In [1]:
# function BuildTree:
#     If every item in the dataset is in the same class
#     or there is no feature left to split the data:
#         return a leaf node with the class label
#     Else:
#         find the best feature and value to split the data
#         split the dataset
#         create a node
#         for each split
#             call BuildTree and add the result as a child of the node
#         return node

In [24]:
import pandas as pd
import numpy as np
import math
from collections import Counter
from TreeNode import TreeNode


class DecisionTree(object):
    '''
    A decision tree class.
    '''

    def __init__(self, impurity_criterion='entropy'):
        '''
        Initialize an empty DecisionTree.
        '''

        self.root = None  # root Node
        self.feature_names = None  # string names of features (for interpreting
                                   # the tree)
        self.categorical = None  # Boolean array of whether variable is
                                 # categorical (or continuous)
        self.impurity_criterion = self._entropy \
                                  if impurity_criterion == 'entropy' \
                                  else self._gini

    def fit(self, X, y, feature_names=None):
        '''
        INPUT:
            - X: 2d numpy array
            - y: 1d numpy array
            - feature_names: numpy array of strings
        OUTPUT: None

        Build the decision tree.
        X is a 2 dimensional array with each column being a feature and each
        row a data point.
        y is a 1 dimensional array with each value being the corresponding
        label.
        feature_names is an optional list containing the names of each of the
        features.
        '''

        if feature_names is None or len(feature_names) != X.shape[1]:
            self.feature_names = np.arange(X.shape[1])
        else:
            self.feature_names = feature_names

        # Create True/False array of whether the variable is categorical
        is_categorical = lambda x: isinstance(x, str) or \
                                   isinstance(x, bool) or \
                                   isinstance(x, str)
        self.categorical = np.vectorize(is_categorical)(X[0])

        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y):
        '''
        INPUT:
            - X: 2d numpy array
            - y: 1d numpy array
        OUTPUT:
            - TreeNode

        Recursively build the decision tree. Return the root node.
        '''

        node = TreeNode()
        index, value, splits = self._choose_split_index(X, y)

        if index is None or len(np.unique(y)) == 1:
            node.leaf = True
            node.classes = Counter(y)
            node.name = node.classes.most_common(1)[0][0]
        else:
            X1, y1, X2, y2 = splits
            node.column = index
            node.name = self.feature_names[index]
            node.value = value
            node.categorical = self.categorical[index]
            node.left = self._build_tree(X1, y1)
            node.right = self._build_tree(X2, y2)
        return node

    def _entropy(self, y):
        '''
        INPUT:
            - y: 1d numpy array
        OUTPUT:
            - float

        Return the entropy of the array y.
        '''

        d = Counter(y)    
        l = len(y)
        ent = 0

        for key, val in d.iteritems():
            p = val / float(l)
            ent += p*np.log2(p)
        return -ent

    def _gini(self, y):
        '''
        INPUT:
            - y: 1d numpy array
        OUTPUT:
            - float

        Return the gini impurity of the array y.
        '''

        d = Counter(y)    
        l = len(y)
        gini = 0

        for key, val in d.iteritems():
            p = val / float(l)
            gini += p*(1-p)
        return gini

    def _make_split(self, X, y, split_index, split_value):
        '''
        INPUT:
            - X: 2d numpy array
            - y: 1d numpy array
            - split_index: int (index of feature)
            - split_value: int/float/bool/str (value of feature)
        OUTPUT:
            - X1: 2d numpy array (feature matrix for subset 1)
            - y1: 1d numpy array (labels for subset 1)
            - X2: 2d numpy array (feature matrix for subset 2)
            - y2: 1d numpy array (labels for subset 2)

        Return the two subsets of the dataset achieved by the given feature and
        value to split on.

        Call the method like this:
        >>> X1, y1, X2, y2 = self._make_split(X, y, split_index, split_value)

        X1, y1 is a subset of the data.
        X2, y2 is the other subset of the data.
        '''

        X1 = []
        X2 = []

        Y1 = []
        Y2 = []
        
        if type(split_value) in ['int', 'float']:
            ind = 0
            for arr in X:
                if arr[split_index] >= split_value:
                    stuff = np.hstack((arr[:split_index], arr[split_index+1:]))
                    X1.extend([stuff])
                    Y1.append(y[ind])
                else:
                    stuff = np.hstack((arr[:split_index], arr[split_index+1:]))
                    X2.extend([stuff])
                    Y2.append(y[ind])
                ind += 1
        else:
            ind = 0
            for arr in X:
                if arr[split_index] == split_value:
                    stuff = np.hstack((arr[:split_index], arr[split_index+1:]))
                    X1.extend([stuff])
                    Y1.append(y[ind])
                else:
                    stuff = np.hstack((arr[:split_index], arr[split_index+1:]))
                    X2.extend([stuff])
                    Y2.append(y[ind])
                ind += 1

        return X1, Y1, X2, Y2

    def _information_gain(self, y, y1, y2):
        '''
        INPUT:
            - y: 1d numpy array
            - y1: 1d numpy array (labels for subset 1)
            - y2: 1d numpy array (labels for subset 2)
        OUTPUT:
            - float

        Return the information gain of making the given split.

        Use self.impurity_criterion(y) rather than calling _entropy or _gini
        directly.
        
        '''
        
        l = float(len(y))
        ig = self._entropy(y) - ((len(y1) / l)  * self._entropy(y1)) - ((len(y2) / l) * self._entropy(y2))

        return ig
    
    def _choose_split_index(self, X, y):
        '''
        INPUT:
            - X: 2d numpy array
            - y: 1d numpy array
        OUTPUT:
            - index: int (index of feature)
            - value: int/float/bool/str (value of feature)
            - splits: (2d array, 1d array, 2d array, 1d array)

        Determine which feature and value to split on. Return the index and
        value of the optimal split along with the split of the dataset.

        Return None, None, None if there is no split which improves information
        gain.

        Call the method like this:
        >>> index, value, splits = self._choose_split_index(X, y)
        >>> X1, y1, X2, y2 = splits
        '''
        col_index = np.array(X).shape[1]    
        igs = []c

        for index in range(col_index):
            uniques = np.unique(X[:, index])
            for unique in uniques:
                X1, y1, X2, y2 = self._make_split(X, y, index, unique)
                ig = self._information_gain(y, y1, y2)
                igs.append((ig, index, unique))

        max_set = sorted(igs, key = lambda x: x[0], reverse = True)[0]

        index = max_set[1]
        value = max_set[2]
        splits = self._make_split(X, y, index, value)

        return index, value, splits

    def predict(self, X):
        '''
        INPUT:
            - X: 2d numpy array
        OUTPUT:
            - y: 1d numpy array

        Return an array of predictions for the feature matrix X.
        '''

        return np.array([self.root.predict_one(row) for row in X])

    def __str__(self):
        '''
        Return string representation of the Decision Tree.
        '''
        return str(self.root)


In [25]:
df = pd.read_csv('../data/playgolf.csv')
df.head()

X = df.drop('Result', axis = 1).as_matrix()
y = df['Result'].as_matrix()

# c_split_index
# c_split_value

In [26]:
tree = DecisionTree()
tree.fit(X, y, df.columns[:-1])
print tree
# y_predict = tree.predict(X)

TypeError: list indices must be integers, not tuple

In [105]:
from collections import Counter
y = np.array([1,1,2,2,2,3,3,3,3,3,3])

def entropy(y):
    '''
    INPUT:
        - y: 1d numpy array
    OUTPUT:
        - float

    Return the entropy of the array y.
    '''
    d = Counter(y)    
    l = len(y)
    ent = 0
    
    for key, val in d.iteritems():
        p = val / float(l)
        ent += p*np.log2(p)
    return -ent

In [106]:
entropy(y)

1.4353713907745331

In [107]:
def gini(y):
    '''
    INPUT:
        - y: 1d numpy array
    OUTPUT:
        - float

    Return the gini impurity of the array y.
    '''
    d = Counter(y)    
    l = len(y)
    gini = 0
    
    for key, val in d.iteritems():
        p = val / float(l)
        gini += p*(1-p)
    return gini

In [108]:
gini(y)

0.5950413223140496

In [109]:
np.random.seed(5)

X = np.random.rand(y.shape[0], 4)

split_index = np.random.randint(0, X.shape[1])

split_value = .5

def _make_split(X, y, split_index, split_value):
    '''
    INPUT:
        - X: 2d numpy array
        - y: 1d numpy array
        - split_index: int (index of feature)
        - split_value: int/float/bool/str (value of feature)
    OUTPUT:
        - X1: 2d numpy array (feature matrix for subset 1)
        - y1: 1d numpy array (labels for subset 1)
        - X2: 2d numpy array (feature matrix for subset 2)
        - y2: 1d numpy array (labels for subset 2)

    Return the two subsets of the dataset achieved by the given feature and
    value to split on.

    Call the method like this:
    >>> X1, y1, X2, y2 = self._make_split(X, y, split_index, split_value)

    X1, y1 is a subset of the data.
    X2, y2 is the other subset of the data.
    '''
    X1 = []
    X2 = []
    
    Y1 = []
    Y2 = []
    
    ind = 0
    for arr in X:
        if arr[split_index] >= split_value:
            stuff = np.hstack((arr[:split_index], arr[split_index+1:]))
            X1.extend([stuff])
            Y1.append(y[ind])
        else:
            stuff = np.hstack((arr[:split_index], arr[split_index+1:]))
            X2.extend([stuff])
            Y2.append(y[ind])
        ind += 1
    
    return X1, Y1, X2, Y2
    
_make_split(X, y, split_index, split_value)

([array([ 0.48841119,  0.61174386,  0.51841799]),
  array([ 0.44130922,  0.15830987,  0.27408646]),
  array([ 0.41423502,  0.29607993,  0.57983781]),
  array([ 0.00164217,  0.51547261,  0.9856244 ]),
  array([ 0.2590976 ,  0.80249689,  0.92274961])],
 [1, 2, 2, 3, 3],
 [array([ 0.22199317,  0.87073231,  0.91861091]),
  array([ 0.2968005 ,  0.18772123,  0.7384403 ]),
  array([ 0.5999292 ,  0.26581912,  0.25358821]),
  array([ 0.32756395,  0.1441643 ,  0.96393053]),
  array([ 0.96022672,  0.18841466,  0.20455555]),
  array([ 0.69984361,  0.77951459,  0.57766286])],
 [1, 2, 3, 3, 3, 3])

In [110]:
X1, y1, X2, y2 = self._make_split(X, y, split_index, split_value)

NameError: name 'self' is not defined

In [134]:
def _information_gain(y, y1, y2):
    '''
    INPUT:
        - y: 1d numpy array
        - y1: 1d numpy array (labels for subset 1)
        - y2: 1d numpy array (labels for subset 2)
    OUTPUT:
        - float

    Return the information gain of making the given split.

    Use self.impurity_criterion(y) rather than calling _entropy or _gini
    directly.
    '''
    l = float(len(y))
    ig = entropy(y) - ((len(y1) / l)  * entropy(y1)) - ((len(y2) / l) * entropy(y2))
    
    return ig

In [169]:
def _choose_split_index(X, y):
    '''
    INPUT:
        - X: 2d numpy array
        - y: 1d numpy array
    OUTPUT:
        - index: int (index of feature)
        - value: int/float/bool/str (value of feature)
        - splits: (2d array, 1d array, 2d array, 1d array)

    Determine which feature and value to split on. Return the index and
    value of the optimal split along with the split of the dataset.

    Return None, None, None if there is no split which improves information
    gain.

    Call the method like this:
    >>> index, value, splits = self._choose_split_index(X, y)
    >>> X1, y1, X2, y2 = splits
    '''

    col_index = X.shape[1]    
    igs = []
    
    for index in range(col_index):
        uniques = np.unique(X[:, index])
        for unique in uniques:
            X1, y1, X2, y2 = _make_split(X, y, index, unique)
            ig = _information_gain( y, y1, y2)
            igs.append((ig, index, unique))
    
    max_set = sorted(igs, key = lambda x: x[0], reverse = True)[0]

    index = max_set[1]
    value = max_set[2]
    splits = _make_split(X, y, index, value)
    
    return index, value, splits


In [170]:
_choose_split_index(X, y)

(1,
 0.51547261190539351,
 ([array([ 0.22199317,  0.20671916,  0.91861091]),
   array([ 0.48841119,  0.76590786,  0.51841799]),
   array([ 0.69984361,  0.02293309,  0.57766286]),
   array([ 0.00164217,  0.63979518,  0.9856244 ]),
   array([ 0.2590976 ,  0.87048309,  0.92274961])],
  [1, 1, 3, 3, 3],
  [array([ 0.2968005 ,  0.08074127,  0.7384403 ]),
   array([ 0.44130922,  0.87993703,  0.27408646]),
   array([ 0.41423502,  0.62878791,  0.57983781]),
   array([ 0.5999292 ,  0.28468588,  0.25358821]),
   array([ 0.32756395,  0.16561286,  0.96393053]),
   array([ 0.96022672,  0.02430656,  0.20455555])],
  [2, 2, 2, 3, 3, 3]))

In [ ]:
Temperature,Humidity,Result
85,85,Don't Play
80,90,Don't Play
83,78,Play
70,96,Play
68,80,Play
65,70,Don't Play
64,65,Play
72,95,Don't Play
69,70,Play
75,80,Play
75,70,Play
72,90,Play
81,75,Play
71,80,Don't Play

In [ ]:
Outlook,Windy,Result
sunny,false,Don't Play
sunny,true,Don't Play
overcast,false,Play
rain,false,Play
rain,false,Play
rain,true,Don't Play
overcast,true,Play
sunny,false,Don't Play
sunny,false,Play
rain,false,Play
sunny,true,Play
overcast,true,Play
overcast,false,Play
rain,true,Don't Play


In [ ]:
df[:,0].unique
->> [sunny, overcast, rain]

In [128]:
np.unique(X[:,0])

array([ 0.00164217,  0.22199317,  0.2590976 ,  0.2968005 ,  0.32756395,
        0.41423502,  0.44130922,  0.48841119,  0.5999292 ,  0.69984361,
        0.96022672])